In [1]:
import pandas as pd
import requests
import time
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.feature_extraction import stop_words

In [3]:
params_happy = {
    "subreddit" : 'happy',
    "size" : 1000
}
params_angry = {
    "subreddit" : 'angry',
    "size" : 1000
}
params_wow = {
    "subreddit" : 'wow',
    "size" : 1000
}

happy_url = 'https://api.pushshift.io/reddit/comment/search/?'
angry_url = 'https://api.pushshift.io/reddit/submission/search/?'
wow_url = 'https://api.pushshift.io/reddit/comment/search/?'

In [6]:
res = requests.get(happy_url,params_happy)
if res.status_code != 200:
    print(f'Error Code: {res.status_code}')
else:
    happy_df = pd.DataFrame(res.json()['data'])

res = requests.get(angry_url,params_angry)
if res.status_code != 200:
    print(f'Error Code: {res.status_code}')
else:
    angry_df = pd.DataFrame(res.json()['data'])

res = requests.get(wow_url,params_wow)
if res.status_code != 200:
    print(f'Error Code: {res.status_code}')
else:
    wow_df = pd.DataFrame(res.json()['data'])

In [7]:
happy_df = happy_df[(~happy_df['body'].str.contains('Thank you for your submission but'))&(~happy_df['body'].str.contains('Thank you for posting to r/happy.'))]
happy_df = happy_df.loc[:,['body','subreddit_id']]
happy_df = happy_df.rename(columns={'subreddit_id':'subreddit'})
happy_df['subreddit'] = 'happy'
print(happy_df.shape)
happy_df.head()

(959, 2)


,body,subreddit
0,Congratulations! Welcome to Parenthood! I wis...,happy
1,"Congratulations! There is no greater blessing,...",happy
2,Indeed. It’s the small things that make a diff...,happy
3,So happy for all of you!,happy
4,"Oh man, I tutored for a couple years in colleg...",happy


In [8]:
angry_df = angry_df[angry_df['selftext']!='[removed]']
angry_df = angry_df.loc[:,['selftext','subreddit']]
angry_df = angry_df.rename(columns = {'selftext':'body'})
print(angry_df.shape)
angry_df.head()

(814, 2)


,body,subreddit
0,People have a habbit of not saying what the re...,angry
1,"Honestly, I don’t even know how to organize th...",angry
2,It's fucking insensitive\nHis 13yr old daughte...,angry
3,,angry
4,"I know since the cause wasn't determined, ther...",angry


In [9]:
final = pd.concat([happy_df,angry_df],axis=0)
final.to_csv('final.csv',index=False)

In [11]:
wow_df = wow_df.loc[:,['author_flair_richtext','score','body','subreddit']]
wow_df.to_csv('wow.csv',index=False)